<a href="https://colab.research.google.com/github/Stuna-alice/Stuna-TestGO/blob/main/ExportCorrect-.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnx

!pip install tensorflow-addons
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e
!apt-get -qq install -y libfluidsynth1
!pip install torchvision
!pip install models

In [11]:
# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)



from google.colab import drive
import os
import zipfile
import torch
import sys


# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'

os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Check if the line contains a colon
            if ':' in line:
                key, value = line.strip().split(':', 1)
                parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"'{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Print all parameters
    print("All Parameters:")
    for key, value in parameters.items():
        print(f"{key}: {value}")

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:
        # Train YOLOv5s with user parameters
        weights_path = parameters['weights']
        !python train.py --img {parameters['img']} --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {weights_path} --cache

        sys.path.append('/content/gdrive/MyDrive/Winforms_data/yolov5/models')
        # Load the YOLOv5 model from yolov5s.yaml
        model_config_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/models/yolov5s.yaml'
        from models.yolo import Model
        model = Model(model_config_path)  # Replace with the actual code to instantiate the YOLOv5 model

        model_name = f"{base_name}.pth"
        weights_dir = os.path.join(working_directory, 'weights')

        # Create the 'weights' directory if it doesn't exist
        os.makedirs(weights_dir, exist_ok=True)

        model_path = os.path.join(weights_dir, model_name)
        torch.save(model.state_dict(), model_path)
        # Convert the .pth file to ONNX format
        onnx_path = f'{working_directory}/zipfiles/{base_name}.onnx'
        dummy_input = torch.randn(int(parameters['batch']), 3, int(parameters['img']), int(parameters['img']))
        torch.onnx.export(model, dummy_input, onnx_path)

        print(f"Model saved as '{model_name}' and exported to '{onnx_path}'.")


In [ ]:
#backup

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os
import zipfile
from pathlib import Path
import shutil
# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Add a print statement to check the content of each line
            print(f"Line read from file: {line}")

            # Check if the line contains the expected delimiter (":")
            if ':' in line:
                key, value = line.strip().split(':')
                parameters[key.strip()] = value.strip()
            else:
                # Print a warning if the line doesn't contain the expected delimiter
                print(f"Warning: Line does not contain the expected delimiter (':'): {line}")

    return parameters
# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/

nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:
        # Determine the appropriate weights file based on the image size
        image_size = int(parameters['img'])
        if image_size == 640:
            weights_file = 'yolov5s.pt'
        elif image_size == 1280:
            weights_file = 'yolov5n6.pt'
        else:
            print(f"Error: Unsupported image size {image_size}. Please use 640 or 1280.")
            weights_file = None

        if weights_file:
            # Train YOLOv5 with user parameters
            !python train.py --img {parameters['img']} --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {weights_file} --cache

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Change directory to allow Google to access GDrive
%cd /content/gdrive/MyDrive/Winforms_data

# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Change directory to the YOLOv5 repository
%cd yolov5

# Install required packages
%pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()  # checks
import zipfile
import os


# Define the path to the zip file and the destination folder
zipFile_name = 'chickenDB.zip'
zip_file_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/zipfiles/' + zipFile_name
extracted_folder_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract'

# Create the destination folder if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zipFile_name}' extracted to '{extracted_folder_path}'.")


# make label code
#training
# Train YOLOv5s on marmot for 3 epochs
!python train.py --img 640  --batch 20 --epochs 10 --data controlDB.yaml --weights yolov5s.pt --cache



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Winforms_data
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/gdrive/MyDrive/Winforms_data/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.1/589.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 36.2 MB/s eta 0:00:00
Checking setup...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-ae5d75c58e79>", line 18, in <cell line: 18>
    display = utils.notebook_init()  # checks
  File "/content/gdrive/MyDrive/Winforms_data/yolov5/utils/__init__.py", line 57, in notebook_init
    from ultralytics.utils.checks import check_requirements
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/__init__.py", line 5, in <module>
    from ultralytics.data.explorer.explorer import Explorer
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/data/explorer/__init__.py", line 3, in <module>
    from .utils import plot_query_result
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/data/explorer/utils.py", line 8, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.10/dist-packages/pandas/__init__.py", line 48, in <module>
    from

TypeError: object of type 'NoneType' has no len()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Change directory to allow Google to access GDrive
%cd /content/gdrive/MyDrive/Winforms_data

# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Change directory to the YOLOv5 repository
%cd yolov5

# Install required packages
%pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()  # checks
import zipfile
import os

# Define the zip file name
zip_file_name = 'chickenDB.zip'  # Change this to the actual zip file name

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: /content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract/{base_name}/images/train/
val: /content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'/content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'/content/gdrive/MyDrive/Winforms_data/yolov5/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Make label code
# training
# Train YOLOv5s on marmot for 3 epochs
!python train.py --img 640 --batch 2 --epochs 2 --data {yaml_file_path} --weights yolov5s.pt --cache


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os
import zipfile

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            key, value = line.strip().split(':')
            parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:

        # Train YOLOv5s with user parameters
        !python train.py --img {parameters['img']}  --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {parameters['weights']} --cache




Mounted at /content/gdrive
 'chickenDB.zip' extracted to '/content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract'.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 3.6 MB/s eta 0:00:00
requirements: Ultralytics requirement ['gitpython'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 29.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 9.3s, installed 1 package: ['gitpython']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

train: weights=yolov5s.pt, cfg=, data=/content/gdrive/MyDrive/Winforms_data/yolov5/data/defaultYamlpath.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn